In [4]:
import pandas as pd
import math
import sys
from statistics import mean

In [5]:
# constants
sample_size = 31
sample_radius = math.floor(sample_size / 2)
cols = ['ch0', 'ch1', 'ch2', 'ch3']

In [31]:
# find stable high and low
def findHighLow(ds):
    freq_count = ds.value_counts(normalize=True)

    indexes = []
    index = 0
    ratio = 0
    mix_avg = 0
    for voltage, freq in freq_count.iteritems():
        ratio = ratio + freq
        mix_avg += voltage
        indexes.append(index)
        if ratio > 0.7:
            break
        index += 1
    mix_avg /= (index + 1)
    # print(mix_avg)

    high = 0
    high_portion = 0
    low = 0
    low_portion = 0
    for voltage, freq in freq_count.iteritems():
        # print(voltage, freq)
        if index < 0:
            break
        if voltage > mix_avg:
            high += voltage * freq
            high_portion += freq
        else:
            low += voltage * freq
            low_portion += freq
        index -= 1
    return high / high_portion, low / low_portion

In [34]:
# crop the pick up and drop down segments
def crop(dirname, fileno):
    # step 0: read input file
    input_file_path = dirname + '/' + str(fileno) + '.csv'
    df = pd.read_csv(input_file_path, usecols=cols)
    # get middle from high, low
    total_rows = df.shape[0]
    high, low = findHighLow(df.round(6)[cols[0]])
    middle = (high + low) / 2.0

    # step 1:
    picked_prev = False
    changing_points = []
    for i in range(total_rows):
        picked_curr = df.loc[i][0] < middle
        if bool(picked_prev) ^ bool(picked_curr):
            changing_points.append(i)
            picked_prev = picked_curr
    # print(changing_points)

    total_pick_drop = len(changing_points)
    print('Total ' + str(total_pick_drop) + ' picks&drops, ')
    
    # step 2:
    indexes = []
    fluctuation = []
    changing_points.append(sys.float_info.max)
    for i in range(1, total_pick_drop):
        changing_point = changing_points[i - 1]
        fluctuation.append(changing_point)
        if changing_points[i] - changing_point > sample_radius: 
            changing_point = math.floor(mean(fluctuation))
            fluctuation = []
            # indexes.append(changing_point)
            indexes.append([changing_point - sample_radius, changing_point + sample_radius])
    # print(indexes)

    # step 3:
    parsed = pd.DataFrame()
    for index in indexes:
        parsed = parsed.append(pd.DataFrame(df.loc[index[0]:index[1]]))
    file_output_path = 'filtered_' + input_file_path
    parsed.to_csv(file_output_path)
    print('output to ' + file_output_path + '\n')

In [36]:
for i in range(17):
    crop('chocolate', i)

Total 39 picks&drops, 
output to filtered_chocolate/0.csv

Total 46 picks&drops, 
output to filtered_chocolate/1.csv

Total 54 picks&drops, 
output to filtered_chocolate/2.csv

Total 52 picks&drops, 
output to filtered_chocolate/3.csv

Total 54 picks&drops, 
output to filtered_chocolate/4.csv

Total 61 picks&drops, 
output to filtered_chocolate/5.csv

Total 75 picks&drops, 
output to filtered_chocolate/6.csv

Total 65 picks&drops, 
output to filtered_chocolate/7.csv

Total 74 picks&drops, 
output to filtered_chocolate/8.csv

Total 60 picks&drops, 
output to filtered_chocolate/9.csv

Total 54 picks&drops, 
output to filtered_chocolate/10.csv

Total 74 picks&drops, 
output to filtered_chocolate/11.csv

Total 62 picks&drops, 
output to filtered_chocolate/12.csv

Total 70 picks&drops, 
output to filtered_chocolate/13.csv

Total 77 picks&drops, 
output to filtered_chocolate/14.csv

Total 66 picks&drops, 
output to filtered_chocolate/15.csv

Total 70 picks&drops, 
output to filtered_chocolat